In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 

In [2]:
import warnings
warnings.filterwarnings('ignore')

### Dataset на 1м записей мой компьютер не потянул, взял данные на 100к

#### Подготовка Датасета

In [3]:
movies = pd.read_csv('movies.csv')

In [4]:
ratings = pd.read_csv('ratings.csv')

In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
df = ratings.join(movies.set_index('movieId'), on='movieId')

In [7]:
dataset = pd.DataFrame({
    'uid':df['userId'],
    'mid': df['movieId'],
    'rating':df['rating']
})

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   uid     100836 non-null  int64  
 1   mid     100836 non-null  int64  
 2   rating  100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [9]:
print(ratings['rating'].min())
print(ratings['rating'].max())

0.5
5.0


#### Dataset Surprise

In [10]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset, Reader, accuracy
from surprise.model_selection import train_test_split, cross_validate

In [11]:
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(dataset, reader)

#### Тетстируем без кросс-валидации

In [12]:
set_train, set_test = train_test_split(data, test_size=0.15, random_state=42)

In [13]:
model = KNNWithMeans(k=50, sim_options={'name':'pearson_baseline', 'user_based':True})

In [14]:
model.fit(set_train)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [15]:
set_pred = model.test(set_test)

In [16]:
accuracy.rmse(set_pred, verbose=True)

RMSE: 0.9046


0.9045807559577181

#### Подберем гиперпараметры, заодно проведя кросс-валидацию на 5 фолдах 

In [17]:
from surprise.model_selection.search import RandomizedSearchCV

In [18]:
params = {'k':[50, 51]}

# , 'sim_options':[{'name':'pearson_baseline', 'user_based':True},
#                                              {'name':'pearson_baseline', 'user_based':False}]

In [19]:
# model_1 = KNNWithMeans()
# result = RandomizedSearchCV(model_1, param_distributions = params, cv=5)

### Не понимаю, в чем ошибка. Точнее, как ее решить

#### На наборе в 100к строк симитирую GridSearch на простых циклах

In [20]:
K = [50, 100, 150]
user_based = [True, False]
score = {}

# for k in tqdm(K):
#     for option in user_based:
#         model = KNNWithMeans(k=k, sim_options={'name':'pearson_baseline', 'user_based':option})
#         score = cross_validate(model, data, measures=['rmse'], cv=5, n_jobs=-1)
#         mean_score = score['test_rmse'].mean()
#         print('RMSE = {}; k = {}; user_based = {}'.format(mean_score, k, option))

#### Таким образом тоже комп не дотягивает

#### В таком случаем просто по отдельности посмотрим модели

In [24]:
model = KNNWithMeans(k=50, sim_options={'name':'pearson_baseline', 'user_based':True})
score = cross_validate(model, data, measures=['rmse'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8939  0.8786  0.9019  0.9029  0.8954  0.8945  0.0087  
Fit time          2.17    2.17    2.06    2.11    1.97    2.09    0.08    
Test time         2.87    2.90    2.96    2.82    2.62    2.83    0.12    


In [25]:
model = KNNWithMeans(k=100, sim_options={'name':'pearson_baseline', 'user_based':True})
score = cross_validate(model, data, measures=['rmse'], cv=5, n_jobs=-1, verbose=True)

Evaluating RMSE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8919  0.8978  0.8995  0.8994  0.8907  0.8959  0.0038  
Fit time          2.39    2.42    2.35    2.27    2.04    2.29    0.14    
Test time         3.12    3.19    3.24    3.14    2.71    3.08    0.19    


#### Комп не тянет item-to-item

In [30]:
model = KNNWithMeans(k=20, sim_options={'name':'pearson_baseline', 'user_based':False})
# score = cross_validate(model, data, measures=['rmse'], cv=5, n_jobs=-1, verbose=True)

In [31]:
model = KNNWithMeans(k=50, sim_options={'name':'pearson_baseline', 'user_based':False})
# score = cross_validate(model, data, measures=['rmse'], cv=5, n_jobs=-1, verbose=True)